# Intermediate Tutorial: Composition and Navigation

This tutorial covers composing prompts from smaller pieces, navigating nested structures, and exporting data.

In [ ]:
from t_prompts import prompt

## Nested Prompts

You can build larger prompts by composing smaller `StructuredPrompt` objects together.

In [ ]:
# Build prompts from smaller pieces
system_msg = "You are a helpful assistant."
user_query = "What is Python?"

p_system = prompt(t"{system_msg:system}")
p_user = prompt(t"User: {user_query:query}")

# Compose into larger prompt
p_full = prompt(t"{p_system:sys} {p_user:usr}")

# Renders correctly
print(str(p_full))

## Navigating Nested Structures

Access nested interpolations using chained subscript operations.

In [ ]:
# Navigate the tree
print(f"System message: {p_full['sys']['system'].value}")
print(f"User query: {p_full['usr']['query'].value}")

# When accessing a nested prompt node, it returns a StructuredPrompt
nested_node = p_full["sys"]
print(f"\nNested node type: {type(nested_node).__name__}")
print(f"Has 'system' key: {'system' in nested_node}")
print(f"Rendered: {str(nested_node)}")

## Conversions: !s, !r, !a

t-strings support conversion flags from Python's string formatting.

In [ ]:
text = "Hello\nWorld"

# !s: str() conversion (default)
p_s = prompt(t"{text!s:s}")
print(f"!s: {str(p_s)}")

# !r: repr() conversion
p_r = prompt(t"{text!r:r}")
print(f"!r: {str(p_r)}")

# !a: ascii() conversion
emoji = "Hello 👋"
p_a = prompt(t"{emoji!a:a}")
print(f"!a: {str(p_a)}")

# Conversion is preserved in metadata
print(f"\nConversion for 'r': {p_r['r'].conversion}")

## Mapping Protocol

`StructuredPrompt` implements the mapping protocol: `keys()`, `values()`, `items()`, `get()`, etc.

In [ ]:
name = "Alice"
age = "30"
city = "NYC"

p = prompt(t"Name: {name:n}, Age: {age:a}, City: {city:c}")

# Keys
print(f"Keys: {list(p.keys())}")

# Values (returns interpolation nodes)
print("\nValues:")
for node in p.values():
    print(f"  {node.key}: {node.value}")

# Items
print("\nItems:")
for key, node in p.items():
    print(f"  {key} -> {node.expression} = {node.value}")

# get() with default
print(f"\nget('n'): {p.get('n').value}")
print(f"get('missing'): {p.get('missing')}")

## Exporting to JSON

Use `toJSON()` to export the prompt structure as JSON.

In [ ]:
import json

context = "User is Alice"
instructions = "Be concise"

p = prompt(t"Context: {context:ctx}. {instructions:inst}")

# Export to JSON
json_data = p.toJSON()
print(json.dumps(json_data, indent=2))

## Nested JSON Export

JSON export works recursively for nested prompts.

In [ ]:
# Create nested structure
inner = prompt(t"{context:ctx}")
outer = prompt(t"Outer: {inner:in}. {instructions:inst}")

# Export nested structure
nested_json = outer.toJSON()
print(json.dumps(nested_json, indent=2))

## Inspecting Prompt Structure

You can recursively inspect nested prompt structures.

In [ ]:
from t_prompts import StructuredPrompt


def inspect_prompt(p, indent=0):
    """Recursively inspect a structured prompt."""
    prefix = "  " * indent
    for key, node in p.items():
        if isinstance(node, StructuredPrompt):
            # Node is a nested prompt
            print(f"{prefix}{key}: (nested prompt)")
            inspect_prompt(node, indent + 1)
        else:
            # Node is an interpolation with a string value
            print(f'{prefix}{key}: {node.expression} = "{node.value}"')


inspect_prompt(outer)

## Next Steps

Continue learning:

- **03-ir-visualization.ipynb**: See the widget visualization for IntermediateRepresentation
- **topics/few-shot-prompts.ipynb**: Build few-shot prompts with dynamic keys and lists
- **topics/source-mapping.ipynb**: Bidirectional text ↔ structure mapping
- **topics/dedenting.ipynb**: Write readable multi-line prompts